# Balanced Active Inference - Demonstration



In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from src.data_generation import generate_friedman_data, split_data
from src.models import ActiveInferenceModels, train_predictive_models
from src.sampling_methods import compute_sampling_probabilities
from src.experiment import run_simulation_experiment
from src.visualization import (
    plot_comparison_results,
    plot_uncertainty_distribution,
    plot_sampling_allocation,
    save_results_to_csv
)

# Set random seed for reproducibility
np.random.seed(0)

print("Packages imported successfully!")

## 1. Data Generation

We generate synthetic data using the Friedman nonlinear function:

$$Y = 10\sin(\pi X_1 X_2) + 20(X_3 - 0.5)^2 + 10X_4 + 5X_5 + \epsilon$$

where $X_1, \ldots, X_5 \sim \text{Uniform}(0, 1)$ and $\epsilon \sim N(0, 0.09)$.

In [ ]:
# Generate data
n_samples = 10000
n_features = 10

X, y = generate_friedman_data(
    n_samples=n_samples,
    n_features=n_features,
    noise_std=0.3,
    random_state=0
)

print(f"Generated {n_samples} samples with {n_features} features")
print(f"Response variable statistics:")
print(f"  Mean: {y.mean():.4f}")
print(f"  Std:  {y.std():.4f}")
print(f"  Min:  {y.min():.4f}")
print(f"  Max:  {y.max():.4f}")

### Split into Training and Test Sets

We use 50% of data for training and 50% for testing (inference population).

In [ ]:
# Split data
X_train, X_test, y_train, y_test = split_data(
    X, y,
    test_size=0.5,
    random_state=0
)

print(f"Training set size: {len(y_train)}")
print(f"Test set size: {len(y_test)}")
print(f"True population mean: {y_test.mean():.4f}")

## 2. Train Predictive Models

We train two models:
1. **Label Model**: Predicts the response variable
2. **Error Model**: Estimates prediction uncertainty

In [ ]:
# Initialize models
models = ActiveInferenceModels(
    label_params={
        'n_estimators': 1000,
        'learning_rate': 0.001,
        'max_depth': 7,
        'random_state': 0
    },
    error_params={
        'n_estimators': 1000,
        'learning_rate': 0.001,
        'max_depth': 7,
        'random_state': 0
    }
)

# Train models
print("Training models...")
models.fit(X_train, y_train)
print("Models trained successfully!")

# Generate predictions and uncertainty estimates on test set
y_pred, uncertainty = models.predict(X_test)
error_pred = models.error_model.predict(X_test)

# Evaluate prediction quality
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_test - y_pred))

print(f"\nLabel Model Performance:")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE:  {mae:.4f}")
print(f"\nUncertainty Statistics:")
print(f"  Mean: {uncertainty.mean():.4f}")
print(f"  Std:  {uncertainty.std():.4f}")

### Visualize Uncertainty Distribution

In [ ]:
# Plot uncertainty distribution
plot_uncertainty_distribution(uncertainty, bins=50)

## 3. Sampling Strategy Comparison

We demonstrate active sampling allocation based on uncertainty.

In [ ]:
# Compute sampling probabilities for a specific budget
budget = 0.1  # 10% sampling rate
tau = 0.5  # Balance parameter

inclusion_probs = compute_sampling_probabilities(
    uncertainty,
    budget=budget,
    tau=tau
)

print(f"Sampling budget: {budget * 100:.1f}%")
print(f"Expected sample size: {inclusion_probs.sum():.1f}")
print(f"Inclusion probability range: [{inclusion_probs.min():.4f}, {inclusion_probs.max():.4f}]")

# Visualize sampling allocation
plot_sampling_allocation(uncertainty, inclusion_probs)

## 4. Run Simulation Experiment

We compare four sampling methods across different budgets:
1. Classical Simple Random Sampling (baseline)
2. Uniform Poisson Sampling (baseline)
3. Active Poisson Sampling (baseline)
4. **Cube Active Sampling (our method)**

**Note:** This may take several minutes depending on your hardware.

In [ ]:
# Define budgets to evaluate
budgets = np.arange(0.05, 0.3, 0.02)

# Run simulation (reduce n_trials for faster testing)
print("Starting simulation experiment...")
print(f"Budgets: {budgets}")
print(f"Number of trials per budget: 1000")

results = run_simulation_experiment(
    y_true=y_test,
    y_pred=y_pred,
    uncertainty=uncertainty,
    error_pred=error_pred,
    budgets=budgets,
    n_trials=1000,  # Reduce to 100 for quick testing
    confidence_level=0.95,
    tau=0.5,
    n_jobs=-1  # Use all available cores
)

print("\nSimulation complete!")

### Display Results Summary

In [ ]:
# Display RMSE results
print("RMSE Results:")
print(results['rmse'])

print("\nInterval Width Results:")
print(results['interval_width'])

print("\nCoverage Rate Results:")
print(results['coverage'])

## 5. Visualize Results

Create publication-quality comparison plots.

In [ ]:
# Generate comprehensive comparison plot
plot_comparison_results(
    results,
    output_path='results/comparison.pdf',
    figsize=(15, 5),
    font_size=18,
    dpi=300
)

## 6. Save Results

Export all results to CSV files for further analysis.

In [ ]:
# Save results to CSV
save_results_to_csv(results, output_dir='results')

print("\nAll results saved successfully!")

## 7. Analyze Relative Performance

Compute relative improvement of cube active sampling over baselines.

In [ ]:
# Compute improvement over classical sampling
rmse_df = results['rmse']

improvement_classical = (
    (rmse_df['classical'] - rmse_df['cube_active']) / rmse_df['classical'] * 100
)

improvement_uniform = (
    (rmse_df['uniform'] - rmse_df['cube_active']) / rmse_df['uniform'] * 100
)

print("RMSE Improvement of Cube Active Sampling:")
print(f"\nOver Classical SRS:")
print(f"  Mean: {improvement_classical.mean():.2f}%")
print(f"  Max:  {improvement_classical.max():.2f}%")

print(f"\nOver Uniform Poisson:")
print(f"  Mean: {improvement_uniform.mean():.2f}%")
print(f"  Max:  {improvement_uniform.max():.2f}%")